# Storing station's images in PostgreSQL database

In [3]:
!conda env list

# conda environments:
#
base                     /home/kiara/anaconda3
geo                      /home/kiara/anaconda3/envs/geo
geotmp                   /home/kiara/anaconda3/envs/geotmp
geotmp1               *  /home/kiara/anaconda3/envs/geotmp1



In [8]:
import requests
from bs4 import BeautifulSoup
import os
import io
from PIL import Image
import psycopg2

#Set the proj_lib_path variable to the path where the proj folder is located in my local machine.
proj_lib_path = "/home/kiara/anaconda3/envs/geotmp1/share/proj"

#Added the proj_lib_path to the PROJ_LIB environment variable using the os.environ dictionary.
os.environ['proj_lib'] = proj_lib_path
proj_lib = os.environ['proj_lib']

#Printed the updated value of the PROJ_LIB environment variable 
print(f"New env var value: \nPROJ_LIB={proj_lib:s}")

# Connect to PostgreSQL database
conn = psycopg2.connect(database="env_db", user="env_master", password="M123xyz", host="localhost", port="5432")
print("Database opened successfully")

# Create a table for images
cur = conn.cursor()
cur.execute('''CREATE TABLE IF NOT EXISTS images
               (id SERIAL PRIMARY KEY,
                pid_val INTEGER,
                image BYTEA);''')
print("Table created successfully")

# Loop through PIDVal values and save images to database
for pid_val in range(1, 101):
    try:
        url = f'https://howis.eglv.de/pegel/html/stammdaten_html/MO_StammdatenPegel.php?PIDVal={pid_val}'
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        map_container = soup.find('div', {'id': 'mapcontainer'})
        img_tag = map_container.find('img')
        ext_url = img_tag['src'][5:] if img_tag else None  # Check if img_tag exists before accessing its 'src' attribute
        
        if ext_url:
            base_url = "https://howis.eglv.de/pegel"
            full_url = base_url + ext_url
            response = requests.get(full_url, stream=True)
            img_data = response.content
            
            # Insert image data into the database
            cur.execute("INSERT INTO images (pid_val, image) VALUES (%s, %s);", (pid_val, psycopg2.Binary(img_data)))
            print(f"Image for PIDVal {pid_val} saved successfully")
        else:
            print(f"No image found for PIDVal {pid_val}")
    except Exception as e:
        print(f"Error while processing PIDVal {pid_val}: {e}")

# Commit the changes and close the connection
conn.commit()
conn.close()


New env var value: 
PROJ_LIB=/home/kiara/anaconda3/envs/geotmp1/share/proj
Database opened successfully
Table created successfully
No image found for PIDVal 1
Image for PIDVal 2 saved successfully
Image for PIDVal 3 saved successfully
Image for PIDVal 4 saved successfully
Image for PIDVal 5 saved successfully
Image for PIDVal 6 saved successfully
No image found for PIDVal 7
Image for PIDVal 8 saved successfully
Image for PIDVal 9 saved successfully
No image found for PIDVal 10
Image for PIDVal 11 saved successfully
Image for PIDVal 12 saved successfully
No image found for PIDVal 13
Image for PIDVal 14 saved successfully
Image for PIDVal 15 saved successfully
No image found for PIDVal 16
Image for PIDVal 17 saved successfully
Image for PIDVal 18 saved successfully
Image for PIDVal 19 saved successfully
Image for PIDVal 20 saved successfully
No image found for PIDVal 21
Image for PIDVal 22 saved successfully
Image for PIDVal 23 saved successfully
Image for PIDVal 24 saved successfully
Im